In [11]:
from osgeo import gdal
import xarray as xr
import rasterio as rx
import numpy as np
from glob import glob
import os
from datetime import datetime as dt
from matplotlib import pyplot as plt

In [2]:
os.chdir('/disk/scratch/local.4/harry/TDX/DEM_timeseries')

In [3]:
DEMs = glob('DEM*HS*.tiff')
DEMs.sort()
DEMs.remove('DEM_TAXI_TDM1_SAR__COS_BIST_HS_S_SRA_20190624T225607_20190624T225608_HH.tiff')
DEMs.remove('DEM_TAXI_TDM1_SAR__COS_BIST_HS_S_SRA_20210630T225621_20210630T225622_HH.tiff')
print('Time Series should have 23 images (all HS in descending orbit)')
len(DEMs)

Time Series should have 23 images (all HS in descending orbit)


23

In [4]:
COHs = glob('*COH*HS*.tiff')
COHs.sort()
print('Same for the coherence files: ')
print(len(COHs))

Same for the coherence files: 
23


In [5]:
datec = [x.split('_')[-2] for x in COHs]
datef = [x.split('_')[-2] for x in DEMs]
print('Are the dates in the coherence files exactly the same as the DEMs?')
datec == datef

Are the dates in the coherence files exactly the same as the DEMs?


True

In [6]:
coarsening = 1 # coarsening factor (set to one as already 3 m DEMs)

def read_timepoint(i):
    # Takes an index i and combines height data and coherence data into a single dataset
    h = xr.open_rasterio(DEMs[i]).sel(band=1).drop('band').rename('height')
    c = xr.open_rasterio(COHs[i]).sel(band=1).drop('band').rename('coherence')
    c = c.interp_like(h)
    ds = xr.merge([h,c]).rename({'x':'lon','y':'lat'}).coarsen(lat=coarsening,lon=coarsening,boundary='trim').mean()
    time = dt.strptime(DEMs[i].split('_')[-3],'%Y%m%dT%H%M%S')
    ds = ds.assign_coords(t=time).expand_dims('t')
    return ds.astype('float16')

def get_dataset():
    timeseries = [read_timepoint(i) for i in range(len(DEMs))]
    LAT,LON = timeseries[0].lat, timeseries[0].lon
    interpolated = [ds.interp(lat=LAT,lon=LON) for ds in timeseries]
    return xr.merge(interpolated)

In [7]:
ds = get_dataset()

/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  """
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  """
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning

/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  """
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  """
/home/s1332488/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning

In [8]:
ds = ds.where(ds>-1) 

In [10]:
ds.to_netcdf('DEM_timeseries_3m.nc')